In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
# Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F4AB178040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F4AB08CCD0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [4]:
llm.invoke("Hello how are you")

AIMessage(content="I am an AI language model, so I don't have feelings or experiences like humans do. However, I'm here and ready to assist you with any questions or tasks you may have. How can I help you today?\n", response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 18, 'total_tokens': 68, 'completion_time': 0.102671818, 'prompt_time': 0.0023992, 'queue_time': None, 'total_time': 0.105071018}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-26ce7a9b-b3a0-4b9e-a2df-c17e70dc1794-0', usage_metadata={'input_tokens': 18, 'output_tokens': 50, 'total_tokens': 68})

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to Japanese"),
    HumanMessage(content="Good morning")
]

llm.invoke(messages)

AIMessage(content='おはようございます (Ohayou gozaimasu) \n', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 23, 'total_tokens': 37, 'completion_time': 0.028, 'prompt_time': 0.00245776, 'queue_time': None, 'total_time': 0.03045776}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-974c4bd1-95f1-4560-9212-210d8ee3f5a9-0', usage_metadata={'input_tokens': 23, 'output_tokens': 14, 'total_tokens': 37})

In [6]:
result = llm.invoke(messages)

In [7]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parser.invoke(result)

'おはようございます (Ohayou gozaimasu) \n'

In [8]:
## Using LCEL, chain the components

chain = llm|parser
chain.invoke(messages)

'おはようございます (ohayo gozaimasu) \n'

In [9]:
## Prompt Templates

from langchain_core.prompts import ChatPromptTemplate


generic_template = "Translate the following from {lang1} to {lang2}:"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", generic_template),
        ("user", "{text}")
    ]
)
prompt

ChatPromptTemplate(input_variables=['lang1', 'lang2', 'text'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lang1', 'lang2'], template='Translate the following from {lang1} to {lang2}:')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))])

In [14]:
prompt.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'],
 'kwargs': {'input_variables': ['lang1', 'lang2', 'text'],
  'messages': [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lang1', 'lang2'], template='Translate the following from {lang1} to {lang2}:')),
   HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]},
 'name': 'ChatPromptTemplate',
 'graph': {'nodes': [{'id': 0, 'type': 'schema', 'data': 'PromptInput'},
   {'id': 1,
    'type': 'runnable',
    'data': {'id': ['langchain', 'prompts', 'chat', 'ChatPromptTemplate'],
     'name': 'ChatPromptTemplate'}},
   {'id': 2, 'type': 'schema', 'data': 'ChatPromptTemplateOutput'}],
  'edges': [{'source': 0, 'target': 1}, {'source': 1, 'target': 2}]}}

In [11]:
result = prompt.invoke({"lang1": "English", "lang2": "Japanese", "text": "Good Morning"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English to Japanese:'), HumanMessage(content='Good Morning')])

In [12]:
result.to_messages()

[SystemMessage(content='Translate the following from English to Japanese:'),
 HumanMessage(content='Good Morning')]

In [13]:
chain = prompt|llm|parser

chain.invoke({"lang1": "English", "lang2": "Japanese", "text": "Good Morning"})

'おはよう (ohayou) \n'